In [102]:
import pandas as pd
import numpy as np

In [103]:
# загрузка датафрейма на 5к строк
df1 = pd.read_csv('IMDB_Movies_Dataset.csv', sep=',')  

# загрузка датафрейма на 130к строк
df2 = pd.read_csv('IMBD.csv', sep=',')  

# обработка столбца с указанием директоров для объединения дф-ов
l = []
for i in df2['director'].to_list():
    if str(i) == 'nan':
        l.append('')
    else:
        l.append(str(i)[2:len(i)-2])
df2['director'] = l  

# объединение дф-ов по полям "Название" и "Средний рейтинг"
df3 = (pd.merge(df1, df2, how = 'inner',
               left_on = ['Title', 'Director'], right_on = ['movie', 'director'])
          .drop_duplicates(['Title', 'Average Rating'])) 

In [104]:
# создание функции для приведения заголовков к единому виду с учётом повторяющихся наименований столбцов
def replace_col_names(columns_names):
    l = list()
    for i in columns_names:
        if i not in l:
            l.append(i.lower().replace(' ', '_'))
        else:
            l.append(i.lower().replace(' ', '_')+'2')
    return l

# приведение заголовков датафрейма к единому формату
df3.columns = replace_col_names(df3.columns)    

# обрезка датафрейма
df3 = df3[['title', 'average_rating', 'director', 'writer',
           'metascore', 'cast', 'release_date', 'country_of_origin', 
           'languages', 'runtime2','genre', 'budget', 'worldwide_gross']] 

# доработка наименований столбцов после обрезки датафрейма
df3 = df3.rename(columns = {'runtime2': 'runtime'}) 

In [105]:
# создание столбца "Год выпуска"
df3['release_year'] = df3['release_date'].apply(lambda x: x.split('(')[0][-5:]) 

# приведение столбца "Год выпуска" к целочисленному типу
df3['release_year'] = df3['release_year'].astype('int')

# удаление столбца с полной датой и страной выпуска
df3 = df3.drop(columns = 'release_date')

In [106]:
# корректировка полей "Бюджет" и "Мировые кассовые сборы" для приведения к числовому виду
df3['budget'] = df3.budget.str.replace("(estimated)", "").str.replace('(', '').str.replace(')', '').str.replace(',','')
df3['worldwide_gross'] = df3.worldwide_gross.str.replace(',','')

# отбор строк, содержащих долларовые суммы, по полям "Бюджет" и "Мировые кассовые сборы" 
for i in ['budget', 'worldwide_gross']:
    ind = df3.columns.get_loc(i)
    for e, j in enumerate(df3[i]):
        if str(j)[0] == '$':
            df3.iloc[e, ind] = str(j)[1:]
        else:
            df3.iloc[e, ind] = None

# удаление строк с курсом, отличающимся от доллара, по полям "Бюджет" и "Мировые кассовые сборы", а также удаление
# пустых значений по полям "cast", "language"
df3 = df3.dropna(subset = ['budget', 'worldwide_gross','cast', 'languages'])

# приведение значений в полях "Бюджет" и "Мировые кассовые сборы" к целочисленному типу
df3['budget'] = df3['budget'].astype('int')
df3['worldwide_gross'] = df3['worldwide_gross'].astype('int')

In [109]:
# очистка поля "Длительность фильма" от единиц измерения (мин.)
df3['runtime'] = df3['runtime'].str.replace('min', '')

# приведение поля "Длительность фильма" к целочисленному типу 
df3['runtime'] = df3['runtime'].astype(pd.Int64Dtype())

In [ ]:
# вывод строк, где в столбце metascore нет пропущенных значений в файл
rows_without_missing_metascore = df3.loc[~df3['metascore'].isna(), ['title', 'average_rating', 'metascore', 'genre']]
rows_without_missing_metascore.to_csv('non_missing_metascore.csv', index=False)

In [112]:
# вывод строк с пропущенными значениями metascore в файл
rows_with_missing_metascore = df3.loc[df3['metascore'].isna(), ['title', 'average_rating', 'metascore', 'genre']]
rows_with_missing_metascore.to_csv('missing_metascore.csv', index=False)

In [113]:
# заполнение пустых значений в поле "Оценка критиков" медианой c группировкой по жанрам
median = df3.groupby('genre')['metascore'].transform('median')
df3['metascore'] = df3['metascore'].fillna(median).fillna(df3['metascore'].median())

In [114]:
# нахождение пустых значений в поле "Длительность фильма" и их заполнение: их было 9, поэтому заполнение ручное
df3[df3['runtime'].isna()]
df3.loc[df3.title == 'Cabrini', 'runtime'] = df3.loc[df3.title == 'Cabrini', 'runtime'].fillna(142)
df3.loc[df3.title == 'Alien: Romulus', 'runtime'] = df3.loc[df3.title == 'Alien: Romulus', 'runtime'].fillna(119)
df3.loc[df3.title == 'Juror #2', 'runtime'] = df3.loc[df3.title == 'Juror #2', 'runtime'].fillna(114)
df3.loc[df3.title == 'The Iron Claw', 'runtime'] = df3.loc[df3.title == 'The Iron Claw', 'runtime'].fillna(132)
df3.loc[df3.title == 'Inside Out 2', 'runtime'] = df3.loc[df3.title == 'Inside Out 2', 'runtime'].fillna(96)
df3.loc[df3.title == 'Transformers One', 'runtime'] = df3.loc[df3.title == 'Transformers One', 'runtime'].fillna(104)
df3.loc[df3.title == 'Conclave', 'runtime'] = df3.loc[df3.title == 'Conclave', 'runtime'].fillna(120)
df3.loc[df3.title == 'Coraline', 'runtime'] = df3.loc[df3.title == 'Coraline', 'runtime'].fillna(100)
df3.loc[df3.title == 'Dune: Part Two', 'runtime'] = df3.loc[df3.title == 'Dune: Part Two', 'runtime'].fillna(160)

# приведение поля "Длительность фильма" к целочисленному типу 
df3['runtime'] = df3['runtime'].astype('int')

In [115]:
# создание столбца "Прибыль/убыток" для определения прибыли/убытка от фильма
df3['profit_or_loss'] = df3['worldwide_gross'] - df3['budget']

# создание столбца "Отношение прибыли/убытка к бюджету фильма" для определения соотношения прибыли/убытка к влож.средствам
df3['p&l_rate'] = round(df3['profit_or_loss']/df3['budget'] * 100, 2)

In [116]:
# проверка итогового вида датафрейма
df3.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 1455 entries, 0 to 3542
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              1455 non-null   object 
 1   average_rating     1455 non-null   float64
 2   director           1455 non-null   object 
 3   writer             1455 non-null   object 
 4   metascore          1455 non-null   float64
 5   cast               1455 non-null   object 
 6   country_of_origin  1455 non-null   object 
 7   languages          1455 non-null   object 
 8   runtime            1455 non-null   int64  
 9   genre              1455 non-null   object 
 10  budget             1455 non-null   int64  
 11  worldwide_gross    1455 non-null   int64  
 12  release_year       1455 non-null   int64  
 13  profit_or_loss     1455 non-null   int64  
 14  p&l_rate           1455 non-null   float64
dtypes: float64(3), int64(5), object(7)
memory usage: 181.9+ KB


In [117]:
# сохранение датафрейма в csv-файл
df3.to_csv('~/final_df.csv.csv')

# сохранение датафрейма в xlsx-файл
df3.to_excel('~/final_df.csv.xlsx')